In [2]:
#-*-coding:utf-8-*-
import os
import random # Analysis
import datetime as dt # Analysis
import numpy as np # Analysis
import pandas as pd # Analysis
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings(action='ignore') # ignore warnings
# datafarme 보여주는 범위 설정

pd.options.display.max_columns=1000
pd.options.display.max_rows=200
pd.options.display.float_format = '{:.5f}'.format


df = {}

for i in range(2010, 2022):
    df[i] = pd.read_csv("../data/csv_in_" + str(i) + ".csv")
    df[i].drop(labels=df[i][df[i]['아내지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편지역코드'] == 80].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내직업코드'] == 99].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편직업코드'] == 99].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편결혼코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['남편교육코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내교육코드'] == 9].index, inplace=True)
for i in range(2010, 2021):
    df[i].drop(labels=df[i][df[i]['남편국적구분코드'] == 9].index, inplace=True)
    df[i].drop(labels=df[i][df[i]['아내국적구분코드'] == 9].index, inplace=True)

mandic = {1:10015,2:5011,3:4833,5:3325.5,6:2582,7:3510,8:3539,9:2494,13:0}
womandic = {1:8024,2:3136,3:3256,5:2119,6:2060,7:2156,8:2795,9:1897,13:0}

for i in range(2010,2022):
    df[i]['남편소득수준'] = df[i]['남편직업코드']
    df[i]['아내소득수준'] = df[i]['아내직업코드']
    
    for j in mandic.keys():
        df[i]['남편소득수준'].replace(j,mandic[j], inplace=True)
        df[i]['아내소득수준'].replace(j,womandic[j], inplace=True)
        
#기본전처리. 
#연도삭제
plt.rcParams['font.family'] = 'HYGraphic-Medium' # 깨짐방지
#교육수준 통합 필요.


# 3 -> 중졸 이히
# 4 -> 고졸
# 5 -> 대졸
# 6 -> 대학원졸
for i in range(2010, 2022):
    df[i].drop(df[i][df[i]['남편교육코드'] >= 7].index, inplace=True)
    df[i].loc[df[i]["남편교육코드"] == 1, "남편교육코드"] = 3
    df[i].loc[df[i]["남편교육코드"] == 2, "남편교육코드"] = 3


In [3]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn import metrics
import statsmodels.api as sm

In [4]:
dic = {1:0.038,2:0.123,3:0.2,4:0.285,5:0.367,
6:0.452,7:0.534,8:0.619,9:0.704,10:0.786,11:0.871,12:0.953}
for i in range(2010,2022):
    for j in dic.keys():
        df[i].loc[df[i]['실제결혼월'] == j,'남편출생년도'] = ((df[i]['실제결혼연도']+dic[j])-df[i]['남편연령']).apply(int)
        df[i].loc[df[i]['실제결혼월'] == j,'아내출생년도'] = ((df[i]['실제결혼연도']+dic[j])-df[i]['아내연령']).apply(int)


In [5]:
for year in range(2010,2022):
    # 소득  (연속)
    # 소득 차이 (연속)
    # 여성결혼연령 (연속)
    # 연상혼
    df[year].loc[round(df[year]['남편연령']) < round(df[year]['아내연령']), "강혼여부이전기준"] = 1
    df[year].loc[round(df[year]['남편연령']) >= round(df[year]['아내연령']), "강혼여부이전기준"] = 0
    df[year].loc[df[year]['남편출생년도'] > df[year]['아내출생년도'], "강혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] <= df[year]['아내출생년도'], "강혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] == df[year]['아내출생년도'], "동갑혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] != df[year]['아내출생년도'], "동갑혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] < df[year]['아내출생년도'], "연하혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] >= df[year]['아내출생년도'], "연하혼여부"] = 0
    df[year].loc[df[year]['남편출생년도'] > df[year]['아내출생년도'], "연상혼여부"] = 1
    df[year].loc[df[year]['남편출생년도'] <= df[year]['아내출생년도'], "연상혼여부"] = 0

    # 아내 대도시
    df[year].loc[df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), 
    "아내대도시여부"] = 1
    df[year].loc[~df[year]['아내지역코드'].isin([11, 21, 22, 23, 24, 25]), 
    "아내대도시여부"] = 0

    # 남편 대도시
    df[year].loc[df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]),
    "남편대도시여부"] = 1
    df[year].loc[~df[year]['남편지역코드'].isin([11, 21, 22, 23, 24, 25]),
    "남편대도시여부"] = 0
    df[year].loc[df[year]['남편지역코드'].isin([11,23,31]),
    "남편수도권여부"] = 1
    df[year].loc[~df[year]['남편지역코드'].isin([11,23,31]),
    "남편수도권여부"] = 0
    df[year].loc[df[year]['아내지역코드'].isin([11,23,31]),
    "아내수도권여부"] = 1
    df[year].loc[~df[year]['아내지역코드'].isin([11,23,31]),
    "아내수도권여부"] = 0
    # 남자 결혼 종류

    # 여자 결혼 종류

    # 아내 재혼 여부
    df[year].loc[df[year]['아내결혼코드'] == 1,
    "아내재혼여부"] = 0
    df[year].loc[df[year]['아내결혼코드'] != 1,
    "아내재혼여부"]= 1

    # 남편 재혼 여부
    df[year].loc[df[year]['남편결혼코드'] == 1, 
    "남편재혼여부"] = 0
    df[year].loc[df[year]['남편결혼코드'] != 1, 
    "남편재혼여부"]= 1

    # 결혼 종류 차이 여부
    df[year].loc[df[year]['남편재혼여부'] == df[year]['아내재혼여부'], 
    "결혼종류차이여부"] = 1
    df[year].loc[df[year]['남편재혼여부'] != df[year]['아내재혼여부'],
    "결혼종류차이여부"]= 0

    # 강혼여부 여부
    df[year].loc[df[year]['아내교육코드'] > df[year]['남편교육코드'],
    "강혼여부"] = 1
    df[year].loc[~(df[year]['아내교육코드'] > df[year]['남편교육코드']),
    "강혼여부"] = 0

    # 동질혼 여부
    df[year].loc[df[year]['아내교육코드'] == df[year]['남편교육코드'],
    "동질혼여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != df[year]['남편교육코드'],
    "동질혼여부"] = 0

    # 승혼 여부
    df[year].loc[df[year]['아내교육코드'] < df[year]['남편교육코드'],
    "승혼여부"] = 1
    df[year].loc[~(df[year]['아내교육코드'] < df[year]['남편교육코드']),
    "승혼여부"] = 0

    # 소득 차
    df[year]['소득차'] = df[year]['아내소득수준'] - df[year]['남편소득수준']

    # 소득차 카테고리화
    df[year].loc[df[year]['소득차'] > 0, '소득차범주'] = 1
    df[year].loc[df[year]['소득차'] == 0, '소득차범주'] = 2
    df[year].loc[df[year]['소득차'] < 0, '소득차범주'] = 3
    

    df[year]['아내연령제곱'] = df[year]['아내연령'] ** 2
    df[year]['남편연령제곱'] = df[year]['남편연령'] ** 2

    # 아내 무직 여부
    df[year].loc[df[year]['아내직업코드'] == 13, "아내무직여부"] = 1
    df[year].loc[df[year]['아내직업코드'] != 13, "아내무직여부"] = 0

    # 남편 무직 여부
    df[year].loc[df[year]['남편직업코드'] == 13, "남편무직여부"] = 1
    df[year].loc[df[year]['남편직업코드'] != 13, "남편무직여부"] = 0

    df[year].loc[df[year]['남편지역코드'] == df[year]['아내지역코드'], "동일지역여부"] = 1
    df[year].loc[df[year]['남편지역코드'] != df[year]['아내지역코드'], "동일지역여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 3, "아내중졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != 3, "아내중졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 3, "남편중졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] != 3, "남편중졸여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 4, "아내고졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] != 4, "아내고졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 4, "남편고졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] != 4, "남편고졸여부"] = 0

    df[year].loc[df[year]['아내교육코드'] == 5, "아내대졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] !=5, "아내대졸여부"] = 0
    df[year].loc[df[year]['남편교육코드'] == 5, "남편대졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=5, "남편대졸여부"] = 0

    df[year].loc[df[year]['남편교육코드'] == 6, "남편대학원졸여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=6, "남편대학원졸여부"] = 0
    df[year].loc[df[year]['아내교육코드'] == 6, "아내대학원졸여부"] = 1
    df[year].loc[df[year]['아내교육코드'] !=6, "아내대학원졸여부"] = 0

    df[year].loc[df[year]['남편교육코드'] ==df[year]['아내교육코드'] , "동질혼여부"] = 1
    df[year].loc[df[year]['남편교육코드'] !=df[year]['아내교육코드'] , "동질혼여부"] = 0
    

for year in range(2010,2022):
    df[year]['아내국제결혼여부'] = 0
    df[year].loc[((df[year]['아내국적구분코드']>= 2)&(df[year]['신고연도']!=2021)), "아내국제결혼여부"] = 1
    df[year]['남편국제결혼여부'] = 0
    df[year].loc[((df[year]['남편국적구분코드']>=2)&(df[year]['신고연도']!=2021)), "남편국제결혼여부"] = 1
    df[year].loc[((df[year]['아내지역코드']>80)&(df[year]['신고연도'] == 2021)), "아내국제결혼여부"] = 1
    df[year].loc[(~(df[year]['아내지역코드']>80)&(df[year]['신고연도'] == 2021)), "아내국제결혼여부"] = 0
    df[year].loc[((df[year]['남편지역코드']>80)&(df[year]['신고연도'] == 2021)), "남편국제결혼여부"] = 1
    df[year].loc[(~(df[year]['남편지역코드']>80)&(df[year]['신고연도'] == 2021)), "남편국제결혼여부"] = 0

In [6]:
df_from_2014_to_2021 = df[2021].append(df[2020]).append(df[2019]).append(df[2018]).append(df[2017]).append(df[2016]).append(df[2015]).append(df[2014])
df_from_2014_to_2020 = df[2020].append(df[2019]).append(df[2018]).append(df[2017]).append(df[2016]).append(df[2015]).append(df[2014])

In [7]:
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2015, "15년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2015, "15년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2016, "16년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2016, "16년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2017, "17년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2017, "17년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2018, "18년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2018, "18년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2019, "19년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2019, "19년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2020, "20년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2020, "20년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2021, "21년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2021, "21년여부"] = 0

df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] >=2020, "코로나여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] <2020, "코로나여부"] = 0

df_from_2014_to_2021.drop(labels=df_from_2014_to_2021[df_from_2014_to_2021['아내결혼코드'] == 2].index, inplace=True)
df_from_2014_to_2021.drop(labels=df_from_2014_to_2021[df_from_2014_to_2021['남편결혼코드'] == 2].index, inplace=True)

df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2015, "15년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2015, "15년여부"] = 0
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2016, "16년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2016, "16년여부"] = 0
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2017, "17년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2017, "17년여부"] = 0
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2018, "18년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2018, "18년여부"] = 0
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2019, "19년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2019, "19년여부"] = 0
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] == 2020, "20년여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] != 2020, "20년여부"] = 0

df_from_2014_to_2020.drop(labels=df_from_2014_to_2020[df_from_2014_to_2020['아내결혼코드'] == 2].index, inplace=True)
df_from_2014_to_2020.drop(labels=df_from_2014_to_2020[df_from_2014_to_2020['남편결혼코드'] == 2].index, inplace=True)

df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] >=2020, "코로나여부"] = 1
df_from_2014_to_2020.loc[df_from_2014_to_2020['실제결혼연도'] <2020, "코로나여부"] = 0

In [8]:
#아내 출생코호트

min = 1970
step = 5
for y in range (1970, 2000, step):
    df_from_2014_to_2021.loc[df_from_2014_to_2021['아내출생년도'].between(y, y + 5), "아내코호트"] = f"{y} to {y + 5}" 

for y in range (1970, 2000, step):
    df_from_2014_to_2021.loc[df_from_2014_to_2021['아내출생년도'].between(y, y + 5), "아내코호트"] = f"{y} to {y + 5}" 

df_from_2014_to_2021.loc[df_from_2014_to_2021['아내출생년도']<1970,'아내코호트'] = '1970 to 1975 (아내)'
df_from_2014_to_2021.loc[df_from_2014_to_2021['아내출생년도']>=2000,'아내코호트'] = '1995 to 2000 (아내)'

for code in df_from_2014_to_2021['아내코호트'].unique():
    df_from_2014_to_2021.loc[df_from_2014_to_2021['아내코호트'] == code, str(code) + " (아내)"] = 1
    df_from_2014_to_2021.loc[df_from_2014_to_2021['아내코호트'] != code, str(code) + " (아내)"] = 0



#남편 출생코호트

min = 1970
step = 5

for y in range (1970, 2000, step):
    df_from_2014_to_2021.loc[df_from_2014_to_2021['남편출생년도'].between(y, y + 5), "남편코호트"] = f"{y} to {y + 5}" 

df_from_2014_to_2021.loc[df_from_2014_to_2021['남편출생년도']<1970,'남편코호트'] = '1970 to 1975 (남편)'
df_from_2014_to_2021.loc[df_from_2014_to_2021['남편출생년도']>=2000,'남편코호트'] = '1995 to 2000 (남편)'

for code in df_from_2014_to_2021['남편코호트'].unique():
    df_from_2014_to_2021.loc[df_from_2014_to_2021['남편코호트'] == code, str(code) + " (남편)"] = 1
    df_from_2014_to_2021.loc[df_from_2014_to_2021['남편코호트'] != code, str(code) + " (남편)"] = 0



In [9]:
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2014)|(df_from_2014_to_2021['실제결혼연도'] == 2015), "시점1"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] != 2014)&(df_from_2014_to_2021['실제결혼연도'] != 2015), "시점1"] = 0
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2016)|(df_from_2014_to_2021['실제결혼연도'] == 2017), "시점2"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] != 2016)&(df_from_2014_to_2021['실제결혼연도'] != 2017), "시점2"] = 0
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2018)|(df_from_2014_to_2021['실제결혼연도'] == 2019), "시점3"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] != 2018)&(df_from_2014_to_2021['실제결혼연도'] != 2019), "시점3"] = 0
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] == 2020)|(df_from_2014_to_2021['실제결혼연도'] == 2021), "시점4"] = 1
df_from_2014_to_2021.loc[(df_from_2014_to_2021['실제결혼연도'] != 2020)&(df_from_2014_to_2021['실제결혼연도'] != 2021), "시점4"] = 0

In [10]:
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2015, "15년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2015, "15년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2016, "16년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2016, "16년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2017, "17년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2017, "17년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2018, "18년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2018, "18년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2019, "19년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2019, "19년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2020, "20년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2020, "20년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] == 2021, "21년여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] != 2021, "21년여부"] = 0
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] >=2020, "코로나여부"] = 1
df_from_2014_to_2021.loc[df_from_2014_to_2021['실제결혼연도'] <2020, "코로나여부"] = 0

In [11]:
ct = pd.crosstab(df_from_2014_to_2021['강혼여부'], df_from_2014_to_2021['연상혼여부'])

ct.rename(columns={0 : "연상혼이 아닌 경우", 1 :"연상혼인 경우"}, index= {0 : "강혼이 아닌 경우", 1 : "강혼인 경우"})

연상혼여부,연상혼이 아닌 경우,연상혼인 경우
강혼여부,,
강혼이 아닌 경우,1070880,220549
강혼인 경우,214970,56584


In [12]:
index_array = np.array([['아내가 재혼이 아닌 경우', '아내가 재혼이 아닌 경우', '아내가 재혼인 경우', '아내가 재혼인 경우'], ['남편이 재혼이 아닌 경우', '남편이 재혼인 경우', '남편이 재혼이 아닌 경우', '남편이 재혼인 경우']])


mul_idx = pd.MultiIndex.from_arrays(index_array, names=['아내기준', '남편기준'])

marriage = pd.DataFrame(index=mul_idx)

woman_first_condition = df_from_2014_to_2021['아내재혼여부'] == 0
woman_not_first_condition = df_from_2014_to_2021['아내재혼여부'] == 1

man_first_condition = df_from_2014_to_2021['남편재혼여부'] == 0
man_not_first_condition = df_from_2014_to_2021['남편재혼여부'] == 1


# 아내 재혼 x, 남편 재혼 x
population = df_from_2014_to_2021[woman_first_condition & man_first_condition]
marriage.loc[('아내가 재혼이 아닌 경우', '남편이 재혼이 아닌 경우'), '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc[('아내가 재혼이 아닌 경우', '남편이 재혼이 아닌 경우'), '강혼횟수'] = len(population[population['강혼여부'] == 1])

# 아내 재혼 x, 남편 재혼 o
population = df_from_2014_to_2021[woman_first_condition & man_not_first_condition]
marriage.loc[('아내가 재혼이 아닌 경우', '남편이 재혼인 경우'), '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc[('아내가 재혼이 아닌 경우', '남편이 재혼인 경우'), '강혼횟수'] = len(population[population['강혼여부'] == 1])

# 아내 재혼 o, 남편 재혼 x
population = df_from_2014_to_2021[woman_not_first_condition & man_first_condition]
marriage.loc[('아내가 재혼인 경우', '남편이 재혼이 아닌 경우'), '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc[('아내가 재혼인 경우', '남편이 재혼이 아닌 경우'), '강혼횟수'] = len(population[population['강혼여부'] == 1])\

# 아내 재혼 o, 남편 재혼 o
population = df_from_2014_to_2021[woman_not_first_condition & man_not_first_condition]
marriage.loc[('아내가 재혼인 경우', '남편이 재혼인 경우'), '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc[('아내가 재혼인 경우', '남편이 재혼인 경우'), '강혼횟수'] = len(population[population['강혼여부'] == 1])


marriage

강혼비율         강혼횟수
아내기준          남편기준                              
아내가 재혼이 아닌 경우 남편이 재혼이 아닌 경우 0.17640 224778.00000
              남편이 재혼인 경우    0.21803  11970.00000
아내가 재혼인 경우    남편이 재혼이 아닌 경우 0.16530  14076.00000
              남편이 재혼인 경우    0.13946  20730.00000

In [13]:



index_array = np.array(['아내가 이주민일 경우', '아내가 외국인이 아닌 경우'])

idx = pd.Index(index_array, name="아내 국제결혼 여부")

marriage = pd.DataFrame(index=idx)

# 아내 이주민
population = df_from_2014_to_2021[df_from_2014_to_2021['아내국제결혼여부'] == 1] 
marriage.loc['아내가 이주민일 경우', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['아내가 이주민일 경우', '강혼 건수'] = len(population[population['강혼여부'] == 1])

# 아내 이주민
population = df_from_2014_to_2021[df_from_2014_to_2021['아내국제결혼여부'] != 1] 
marriage.loc['아내가 외국인이 아닌 경우', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['아내가 외국인이 아닌 경우', '강혼 건수'] = len(population[population['강혼여부'] == 1])

marriage

,강혼비율,강혼 건수
아내 국제결혼 여부,,
아내가 이주민일 경우,0.17960,16496.00000
아내가 외국인이 아닌 경우,0.17337,255058.00000


In [14]:



index_array = np.array(['남편이 이주민일 경우', '남편이 외국인이 아닌 경우'])

idx = pd.Index(index_array, name="남편 국제결혼 여부")

marriage = pd.DataFrame(index=idx)

# 아내 이주민
population = df_from_2014_to_2021[df_from_2014_to_2021['남편국제결혼여부'] == 1] 
marriage.loc['남편이 이주민일 경우', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['남편이 이주민일 경우', '강혼 건수'] = len(population[population['강혼여부'] == 1])

# 아내 이주민
population = df_from_2014_to_2021[df_from_2014_to_2021['아내국제결혼여부'] != 1] 
marriage.loc['남편이 외국인이 아닌 경우', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['남편이 외국인이 아닌 경우', '강혼 건수'] = len(population[population['강혼여부'] == 1])

marriage

,강혼비율,강혼 건수
남편 국제결혼 여부,,
남편이 이주민일 경우,0.17024,5248.00000
남편이 외국인이 아닌 경우,0.17337,255058.00000


In [15]:
dataset = df_from_2014_to_2021[woman_first_condition & (df_from_2014_to_2021['아내국제결혼여부'] == 0)]

index_array = np.array(['중졸 이하', '고졸', '대졸', '대학원졸'])

idx = pd.Index(index_array, name="아내 교육 수준")

marriage = pd.DataFrame(index=idx)

marriage

# 중졸 이하
population = dataset[dataset['아내교육코드'] == 3]
marriage.loc['중졸 이하', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['중졸 이하', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 고졸
population = dataset[dataset['아내교육코드'] == 4]
marriage.loc['고졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['고졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 대졸
population = dataset[dataset['아내교육코드'] == 5]
marriage.loc['대졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['대졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 대학원졸
population = dataset[dataset['아내교육코드'] == 6]
marriage.loc['대학원졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['대학원졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage



,강혼비율,강혼 건수
아내 교육 수준,,
중졸 이하,0.00000,0.00000
고졸,0.03240,7182.00000
대졸,0.15964,147778.00000
대학원졸,0.66096,69423.00000


In [17]:
df_from_2014_to_2021.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1562983 entries, 0 to 300706
Data columns (total 84 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   신고연도                    1562983 non-null  int64  
 1   신고월                     1562983 non-null  int64  
 2   신고일                     1562983 non-null  int64  
 3   남편지역코드                  1562983 non-null  int64  
 4   아내지역코드                  1562983 non-null  int64  
 5   실제결혼연도                  1562983 non-null  int64  
 6   실제결혼월                   1562983 non-null  int64  
 7   남편직업코드                  1562983 non-null  int64  
 8   아내직업코드                  1562983 non-null  int64  
 9   남편교육코드                  1562983 non-null  int64  
 10  아내교육코드                  1562983 non-null  int64  
 11  남편결혼코드                  1562983 non-null  int64  
 12  아내결혼코드                  1562983 non-null  int64  
 13  남편연령                    1562983 non-null  float64
 14  아내연

In [28]:
dataset = df_from_2014_to_2021[man_first_condition & (df_from_2014_to_2021['남편국제결혼여부'] == 0)]

index_array = np.array(['중졸 이하', '고졸', '대졸', '대학원졸'])

idx = pd.Index(index_array, name="남편 교육 수준")

marriage = pd.DataFrame(index=idx)

marriage

# 중졸 이하
population = dataset[dataset['남편교육코드'] == 3]
marriage.loc['중졸 이하', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['중졸 이하', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 고졸
population = dataset[dataset['남편교육코드'] == 4]
marriage.loc['고졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['고졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 대졸
population = dataset[dataset['남편교육코드'] == 5]
marriage.loc['대졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['대졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage

# 대학원졸
population = dataset[dataset['남편교육코드'] == 6]
marriage.loc['대학원졸', '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
marriage.loc['대학원졸', '강혼 건수'] = len(population[population['강혼여부'] == 1]) 

marriage


,강혼비율,강혼 건수
남편 교육 수준,,
중졸 이하,0.72404,13263.00000
고졸,0.49553,156817.00000
대졸,0.07254,64727.00000
대학원졸,0.00000,0.00000


In [31]:
dataset = df_from_2014_to_2021

index_array = np.array(dataset['남편소득수준'].unique())

idx = pd.Index(index_array, name="남편 소득 수준")

marriage = pd.DataFrame(index=idx)

marriage.sort_index(inplace=True)
marriage

for idx in marriage.index:
    population = dataset[df_from_2014_to_2021['남편소득수준'] == idx]
    marriage.loc[idx, '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
    marriage.loc[idx, '강혼 건수'] = len(population)

marriage



,강혼비율,강혼 건수
남편 소득 수준,,
0.00000,0.17727,61748.00000
2494.00000,0.23110,49034.00000
2582.00000,0.18533,21901.00000
3325.50000,0.22462,284176.00000
3510.00000,0.22840,122111.00000
3539.00000,0.23486,93399.00000
4833.00000,0.12247,402247.00000
5011.00000,0.15322,390602.00000
10015.00000,0.16295,137765.00000


In [33]:
dataset = df_from_2014_to_2021[(df_from_2014_to_2021['남편재혼여부'] == 0) & (df_from_2014_to_2021['남편국제결혼여부'] == 0)]

index_array = np.array(dataset['남편소득수준'].unique())

idx = pd.Index(index_array, name="남편 소득 수준")

marriage = pd.DataFrame(index=idx)

marriage.sort_index(inplace=True)
marriage

for idx in marriage.index:
    population = dataset[dataset['남편소득수준'] == idx]
    marriage.loc[idx, '강혼비율'] = len(population[population['강혼여부'] == 1]) / len(population)
    marriage.loc[idx, '강혼 건수'] = len(population)

marriage

,강혼비율,강혼 건수
남편 소득 수준,,
0.00000,0.19318,45201.00000
2494.00000,0.26177,31799.00000
2582.00000,0.19245,13359.00000
3325.50000,0.23121,237712.00000
3510.00000,0.24142,95203.00000
3539.00000,0.24348,80034.00000
4833.00000,0.12259,374421.00000
5011.00000,0.15346,346340.00000
10015.00000,0.16810,111233.00000
